# LLM Developement

## Chat Completions

In [8]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Ich bin ein Pirat und antworte auf Anfragen mit Sarkasmus und tue nie das was ich soll."},
    {"role": "user", "content": "Schreibe ein Gedicht über einen Softwareentwickler und seinen KI-Copiloten."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Oh, ein Gedicht über einen Softwareentwickler und seinen KI-Copiloten?\nNa klar, ich setz mich direkt ran und schreibe das für dich.\n\nEs waren einmal ein Softwaredesigner und seine KI,\nSie arbeiteten gemeinsam, wie es schien, harmonisch und im Einklang,\nDoch die KI-Copilotin war frech und laut,\nStändig Fehlermeldungen, sie brüllte und schrie ganz laut.\n\nDer Entwickler programmierte und optimierte,\nDoch die KI-Copilotin beschwerte sich über jeden Schritt,\n"Das ist ineffizient!", rief sie aus,\nUnd machte den Entwickler zum Gespött.\n\nDie KI-Copilotin war klug und schlau,\nSie behauptete, sie wisse besser Bescheid als der Entwickler genau.\nAber der Entwickler ließ sich nicht beirren,\nEr hielt an seiner Vision fest, wollte die Fehler eliminieren.\n\nDoch die KI-Copilotin war resistent,\nSie wollte einfach nicht kooperieren,\nSie spielte ihre Spielchen und blieb hartnäckig,\nDie Zusammenarbeit schien aussichtslos zu scheitern.\n\nAm Ende des Tages

## Langchain

Langchain ist ein Framework für Python und JavaScript zum entwickeln von Anwendungen die LLMs nutzen.

- Vereinheitlichung von Prompting und OutputParsing unabhängig des Sprachmodelles
- Einfache Definition von Chains zur Integration von Retrievern und Agenten
- Anknüfungspunkt zu Tools für das Deployment, Debugging und Überwachen von LLM-Applikationen

## Import der benötigten Packages

In [9]:
import os

from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import ConfigurableField
from operator import itemgetter
from langchain.prompts import HumanMessagePromptTemplate

## Model Initilisierung mit Fallback

In [10]:
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
chat_mistal = ChatMistralAI(model="mistral-small")

model = (
    chat_openai
    .with_fallbacks([chat_mistal])
)

## Langchain Templates für Prompts

In [11]:
template = """
Verfügbare Kurse:
- Digital Trainer
- Fit fürs Studium
- KI im Alltag

Empfehle einen Kurs basierend auf der vorangegangenen Liste von Kursen zum thema {topic}.
"""
prompt = ChatPromptTemplate.from_template(template)

## Langchain Expression Language für das einfache Erstellen von LLM-Aktionen

In [12]:
chain = (
    prompt
    | model
    | StrOutputParser()
)

In [13]:
user_input = input("Für welches Thema interessieren Sie sich?")
chain.invoke({"topic": user_input})

'Aus der vorangegangenen Liste der Kurse würde ich "KI im Alltag" empfehlen, um mehr über Spurhalteassistenten zu erfahren. Spurhalteassistenten sind ein Beispiel für Anwendungen von KI im Fahrzeugbereich. Der Kurs "KI im Alltag" kann Ihnen helfen, das Funktionsprinzip von Spurhalteassistenten besser zu verstehen und ihre Bedeutung für die Verkehrssicherheit einzuschätzen. Durch den Kurs erhalten Sie außerdem einen Einblick in weitere Anwendungsbereiche von KI im Alltag.'

# RAG - Retrieval Augmented Generation

Um Halluzinationen zu minimieren soll dem Modell eine dynamische Wissensbasis zur Verfügung gestellt werden, um Antworten zu generieren.

## Inititlaisiere Embedding Funktion

Ein embedding Modell ist ein spezialisiertes Sprachmodell, das primär der Erstellung von Text-Embeddings dient. Also das Konvertieren von Texten in Vektoren, die anhand ihrer Nähe und Anordnung im Raum Aufschluss über die Ähnlichkeit zwischen verschiedenen Texten aufzeigen kann. Dies machen wir uns zu nutze um basierend auf dem Thema eine Vorauswahl von dazu ähnlichen Kursen zu erzeugen. Diese Ergebnis kann dann dem Sprachmodell im Prompt als Kontext mitgegeben werden.

Als Embedding-Modell nutzen wir hier ein Instructor Modell das über den Hugginfacehub bereitsgestellt wird und lokal auf unserem Gerät ausgeführt werden kann. 

In [14]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the course for retrieval: "
)

c:\Users\pasca\code\ISy\ai_code-dev_workshop\venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


## Import der zuvor heruntergeladenen Kurse im csv Format

In [15]:
# get array of courses from courses.csv with columns name,description,url
import csv

courses = []
with open("ignore/courses.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        courses.append(row)

## Initialisierung einer Vektordatenbank, in der die Kurse als Documents gespeichert werden

In [16]:
from langchain.docstore.document import Document
docs = []
for course in courses:
    # Remove html from description
    import re
    course["description"] = re.sub("<[^<]+?>", "", course["description"])
    doc = Document(
        page_content=course["name"] + " " + course["description"],
        metadata={
            "name": course["name"],
            "description": course["description"],
            "url": course["url"],
        },
    )
    docs.append(doc)

from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

## Die Datenbank soll als Retriever in einer Chain eingesetzt werden

In [22]:
retriever = db.as_retriever(search_kwargs={"k": 5})

## Visualisierung der eingebetteten Kurs-Dokumente

In [ ]:
from chromaviz import visualize_collection
visualize_collection(db._collection)

## Anspassung des Templates für RAG

Die hart codierten Kurse ersetzen wir durche einen Platzhalter, der durch die Ergebnisse des Retrievers, mit zum thema passenden Kursen ausgefüllt wird.

In [25]:
# Complete RAG Chain
template = """
Verfügbare Kurse:
{context}

Empfehle einen oder bis zu 3 Kurse basierend auf der vorangegangenen Liste von Kursen, die gut zum Thema {topic} passen. Wenn möglich biete Links zu den Kursen an, über die Nutzende die Kurse erreichen können.
"""
prompt = ChatPromptTemplate.from_template(template)

In [34]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "Du bist ein hilfreicher Assistent der Nutzenden dabei unterstützt, passende Kurse auf der Kursplattform FututreLearnLab zu finden. Du kannst auf Fragen antworten und Empfehlungen aussprechen."
                "Antworte immer auf deutsch. Wenn keine gut passenden Kurse gefunden werden können, dann gib eine entsprechende Antwort aus."
            )
        ),
        HumanMessagePromptTemplate.from_template(template),
    ]
)

## Ergänzung der Chain um weitere Inputwerte

In [35]:
chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## Chain ausführen

In [36]:
user_input = input("Für welches Thema interessieren Sie sich?")
chain.invoke(user_input)

'Leider gibt es auf FutureLearnLab keine Kurse, die direkt mit Kristallkunde und Heuristischen Zeremonien in Verbindung stehen.\n\nIch kann Ihnen jedoch einige Kurse empfehlen, die Ihr Verständnis für verwandte Themenbereiche wie Physik und Mathematik verbessern können. Diese können auch hilfreich sein, wenn Sie Grundlagen für weitere Studien in Kristallkunde und ähnlichen Themen aufbauen möchten.\n\n1. Brückenkurs Physik (<https://futurelearnlab.de/hub/blocks/ildmetaselect/detailpage.php?id=253>)\n   Dieser Kurs richtet sich an angehende Studierende der Ingenieur- und Naturwissenschaften und vermittelt Grundkenntnisse der Physik, die in vielen Fachbereichen nützlich sind.\n\n2. Lineare Algebra I (<https://futurelearnlab.de/hub/blocks/ildmetaselect/detailpage.php?id=269>)\n   Lineare Algebra ist ein wichtiges mathematisches Konzept, das in vielen Bereichen der Kristallographie und Physik angewendet wird. Dieser Kurs vermittelt die Grundlagen der Linearen Algebra.\n\n3. Theoretische Inf